# BERT Fine-Tuning for Ticket Classification

Transfer learning ile pre-trained BERT modelini kullanarak accuracy'yi %90+ seviyelerine çıkarabiliriz.

**Beklenen Accuracy: 90-93%**


In [1]:
   import torch
   print(f"GPU Kullanılabilir: {torch.cuda.is_available()}")
   if torch.cuda.is_available():
       print(f"GPU: {torch.cuda.get_device_name(0)}")
       print("🎉 HAZIRSINIZ!")
   else:
       print("⚠️ CUDA kurulumu gerekli")

GPU Kullanılabilir: True
GPU: NVIDIA GeForce RTX 2060
🎉 HAZIRSINIZ!


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import pickle
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


## 1. Load and Prepare Data


In [ ]:
# Load data
df = pd.read_csv("../data/cleaned_data.csv")
print(f"Data shape: {df.shape}")
print(f"\nLabel distribution:\n{df['label'].value_counts()}")

# Train/val/test split
X_train, X_tmp, y_train, y_tmp = train_test_split(
    df["text"], df["label"], test_size=0.2, random_state=42, stratify=df["label"]
)
X_val, X_test, y_val, y_test = train_test_split(
    X_tmp, y_tmp, test_size=0.5, random_state=42, stratify=y_tmp
)

print(f"\nTrain: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

# Label encoding
label_encoder = LabelEncoder()
label_encoder.fit(df['label'])
y_train_encoded = label_encoder.transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

num_labels = len(label_encoder.classes_)
print(f"\nNumber of classes: {num_labels}")
print(f"Classes: {label_encoder.classes_}")


## 2. Create Dataset Class


In [ ]:
class TicketDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

print("Dataset class created!")


## 3. Initialize Tokenizer and Create DataLoaders


In [ ]:
# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
print("Tokenizer loaded!")

# Hyperparameters
MAX_LENGTH = 128
BATCH_SIZE = 16  # Reduced for memory efficiency
EPOCHS = 3
LEARNING_RATE = 2e-5

# Create datasets
train_dataset = TicketDataset(X_train, y_train_encoded, tokenizer, MAX_LENGTH)
val_dataset = TicketDataset(X_val, y_val_encoded, tokenizer, MAX_LENGTH)
test_dataset = TicketDataset(X_test, y_test_encoded, tokenizer, MAX_LENGTH)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")


## 4. Initialize BERT Model


In [ ]:
# Initialize model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=num_labels,
    output_attentions=False,
    output_hidden_states=False
)

model = model.to(device)
print(f"Model loaded and moved to {device}!")
print(f"Number of parameters: {sum(p.numel() for p in model.parameters()):,}")


## 5. Setup Optimizer and Scheduler


In [ ]:
# Optimizer
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, eps=1e-8)

# Total training steps
total_steps = len(train_loader) * EPOCHS

# Learning rate scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

print(f"Optimizer and scheduler configured!")
print(f"Total training steps: {total_steps}")


## 6. Training and Evaluation Functions


In [ ]:
def train_epoch(model, data_loader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    correct_predictions = 0
    
    for batch in tqdm(data_loader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        logits = outputs.logits
        
        total_loss += loss.item()
        
        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
    
    return correct_predictions.double() / len(data_loader.dataset), total_loss / len(data_loader)

def eval_model(model, data_loader, device):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    predictions = []
    true_labels = []
    
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            
            loss = outputs.loss
            logits = outputs.logits
            
            total_loss += loss.item()
            
            _, preds = torch.max(logits, dim=1)
            correct_predictions += torch.sum(preds == labels)
            
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    
    return correct_predictions.double() / len(data_loader.dataset), total_loss / len(data_loader), predictions, true_labels

print("Training and evaluation functions created!")


## 7. Train the Model


In [ ]:
print("=" * 60)
print("BERT FINE-TUNING BAŞLIYOR")
print("=" * 60)

history = {
    'train_acc': [],
    'train_loss': [],
    'val_acc': [],
    'val_loss': []
}

best_val_acc = 0

for epoch in range(EPOCHS):
    print(f'\nEpoch {epoch + 1}/{EPOCHS}')
    print('-' * 60)
    
    train_acc, train_loss = train_epoch(model, train_loader, optimizer, scheduler, device)
    print(f'Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}')
    
    val_acc, val_loss, _, _ = eval_model(model, val_loader, device)
    print(f'Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}')
    
    history['train_acc'].append(train_acc.item())
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc.item())
    history['val_loss'].append(val_loss)
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), '../models/bert_model.pt')
        print(f'*** Model saved! Best Val Acc: {best_val_acc:.4f} ***')

print("\n" + "=" * 60)
print("EĞİTİM TAMAMLANDI!")
print("=" * 60)


## 8. Evaluate on Test Set


In [ ]:
# Load best model
model.load_state_dict(torch.load('../models/bert_model.pt'))

# Evaluate on test set
test_acc, test_loss, test_predictions, test_true_labels = eval_model(model, test_loader, device)

print("=" * 60)
print("TEST SET SONUÇLARI")
print("=" * 60)
print(f"Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"Test Loss: {test_loss:.4f}")

print("\nDetaylı Sınıflandırma Raporu:")
print(classification_report(
    test_true_labels, 
    test_predictions, 
    target_names=label_encoder.classes_,
    zero_division=0
))


## 9. Save Training History and Tokenizer


In [ ]:
# Save training history
with open('../models/bert_training_history.pkl', 'wb') as f:
    pickle.dump(history, f)

# Save tokenizer
tokenizer.save_pretrained('../models/bert_tokenizer')

# Save label encoder (if not already saved)
with open('../models/label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

print("Training history and tokenizer saved!")
print("\nSaved files:")
print("  - ../models/bert_model.pt")
print("  - ../models/bert_training_history.pkl")
print("  - ../models/bert_tokenizer/")
print("  - ../models/label_encoder.pkl")
